In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime

### 1. Check sample dataset

In [4]:
# take a look at a data sample
data = pd.read_csv("raw_data/201710-citibike-tripdata.csv")
data.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,457,2017-10-01 00:00:00,2017-10-01 00:07:38,479,9 Ave & W 45 St,40.760193,-73.991255,478,11 Ave & W 41 St,40.760301,-73.998842,30951,Subscriber,1985.0,1
1,6462,2017-10-01 00:00:20,2017-10-01 01:48:03,279,Peck Slip & Front St,40.707873,-74.001670,307,Canal St & Rutgers St,40.714275,-73.989900,14809,Customer,NaN,0
2,761,2017-10-01 00:00:27,2017-10-01 00:13:09,504,1 Ave & E 16 St,40.732219,-73.981656,350,Clinton St & Grand St,40.715595,-73.987030,28713,Subscriber,1992.0,1
3,1193,2017-10-01 00:00:29,2017-10-01 00:20:22,3236,W 42 St & Dyer Ave,40.758985,-73.993800,3233,E 48 St & 5 Ave,40.757246,-73.978059,16008,Customer,1992.0,2
4,2772,2017-10-01 00:00:32,2017-10-01 00:46:44,2006,Central Park S & 6 Ave,40.765909,-73.976342,469,Broadway & W 53 St,40.763441,-73.982681,14556,Customer,NaN,0


### 2. Data loading, cleaning, and clustering

In [ ]:
y_ind = ['2016','2017','2018','2019'] # list of years interested
m_ind = list() 
for i in range(1,13):
    m_ind.append(str(i)) # list of months

##### Note: Citibike dataset file could be as large as 200M per month. I suggest to start with 1 year and 1 month. 

In [170]:
# initialize empty dfs for each types of analysis
data_user = pd.DataFrame()
data_gender = pd.DataFrame()
data_age = pd.DataFrame()

# loop over all years and months
for y in y_ind:
    for m in m_ind:
        # if month is '1-9', turn it to '01-09'
        if len(m) == 1: 
            m_temp = "0" + m
        else:
            m_temp = m
        
        # try to load data, when finished, print
        try:
            data1 = pd.read_csv("data/" + y + m_temp + "-citibike-tripdata.csv")
        except:
            print('Finish reading all the data')
        
        # print some notes to know how many files we have loaded
        print("---------")
        print(y + m_temp + "-citibike-tripdata.csv")
        
        # Cleaning step 1: uniform column names
        # data between 2016-10 and 2017-02 has different column names from other time. 
        # We want to change them to uniformed names
        try:
            data1['starttime'] # the column name is either 'startime' or 'Start Time'
        except:
            data1.rename(columns={u'Start Time' : 'starttime',\
                     u'Bike ID' : 'bikeid', \
                     u'User Type' : 'usertype',\
                     u'Birth Year' : 'birth year',\
                     u'Gender' : 'gender',\
                      u'Trip Duration' : 'tripduration'}, inplace=True)
            print("Note: Column name is corrected")
            
        # Cleaning step 2: remove NaN values   
        ind = (pd.notna(data1['birth year'])) & (pd.notna(data1['gender'])) & (pd.notna(data1['usertype'])) & (pd.notna(data1['starttime']))
        data_temp = data1.loc[ind].copy()
        s = pd.Series(range(0,len(data_temp.index))) # reorganize index
        data_temp.set_index([s],inplace = True)
        
        # Cleaning step 3: Convert time str to datetime object
        # Citibike data has the following 4 types of time str: 2019-01-31, 1/3/2019, 1/31/2019, 10/31/2019
        # We need to test each one until we get a correct result
        try:
            time = datetime.strptime(data_temp['starttime'][0][0:10], '%Y-%m-%d')
        except:
            try:
                time = datetime.strptime(data_temp['starttime'][0][0:8], '%m/%d/%Y')
            except:
                try:
                    time = datetime.strptime(data_temp['starttime'][0][0:9], '%m/%d/%Y')
                except:
                    time = datetime.strptime(data_temp['starttime'][0][0:10], '%m/%d/%Y')
        
        # Data cleaning finished, let's start to aggregate the data to each type
        
        # 1. generate Usertype data
        data_temp_user = pd.DataFrame()
        data_temp_user['time'] = pd.Series(time)
        # set index to be 2 columns and count along 'usertype'
        count1 = data_temp.set_index(['usertype','tripduration']).count(level = 'usertype').bikeid
        data_temp_user['count_all'] = pd.Series(sum(count1))
        data_temp_user['count_sub'] = pd.Series(count1.Subscriber)
        # for some months, only subscribers ride Citibike
        try:
            data_temp_user['count_cus'] = pd.Series(count1.Customer)
        except:
            data_temp_user['count_cus'] = pd.Series(0)
            print('Note: There is no casual customer for ' + y + '-' + m_temp)
        # add monthly data to a predefined dataframe
        data_user = data_user.append(data_temp_user)
        
        # 2. generate Gender data
        data_temp_gender = pd.DataFrame()
        data_temp_gender['time'] = pd.Series(time)
        count2 = data_temp.set_index(['gender','tripduration']).count(level = 'gender').bikeid
        data_temp_gender['count_all'] = pd.Series(sum(count2))
        data_temp_gender['count_m'] = pd.Series(count2[1]) # male
        data_temp_gender['count_f'] = pd.Series(count2[2]) # female
        data_temp_gender['count_u'] = pd.Series(count2[0]) # unknown
        data_gender = data_gender.append(data_temp_gender)
        
        # 3. generate Age data
        data_temp_age = pd.DataFrame()
        data_temp_age['time'] = pd.Series(time)
        data_temp['age'] = int(y) - data_temp['birth year'] # calculate age
        # split age groups. Only age over 16 is allowed to use Citibike
        age_group = [16,18,22,30,40,50,60,70,max(data_temp.age)]
        # use 'groupby' and 'cut' method to count over age groups
        count3 = data_temp.groupby(pd.cut(data_temp.age, age_group)).count().bikeid        
        data_temp_age['count_all'] = pd.Series(sum(count3))
        # assign result to each age group
        for i in range(0,len(age_group) - 1):
            data_temp_age['count_g' + str(i + 1)] = pd.Series(count3[i])            
        data_age = data_age.append(data_temp_age)       


201601-citibike-tripdata.csv
There is no casual customer for 2016-01
201602-citibike-tripdata.csv
There is no casual customer for 2016-02
201603-citibike-tripdata.csv
There is no casual customer for 2016-03
201604-citibike-tripdata.csv
There is no casual customer for 2016-04
201605-citibike-tripdata.csv
There is no casual customer for 2016-05
201606-citibike-tripdata.csv
There is no casual customer for 2016-06
201607-citibike-tripdata.csv
201608-citibike-tripdata.csv
201609-citibike-tripdata.csv
201610-citibike-tripdata.csv
Column name is corrected
201611-citibike-tripdata.csv
Column name is corrected
201612-citibike-tripdata.csv
Column name is corrected
201701-citibike-tripdata.csv
Column name is corrected
201702-citibike-tripdata.csv
Column name is corrected
201703-citibike-tripdata.csv
Column name is corrected
201704-citibike-tripdata.csv
201705-citibike-tripdata.csv
201706-citibike-tripdata.csv
201707-citibike-tripdata.csv
201708-citibike-tripdata.csv
201709-citibike-tripdata.csv
2

### 3. Take a look at each dataset we have clustered

In [180]:
s1 = pd.Series(range(0,len(data_user.index))) # give new ordered index
data_user.set_index([s1],inplace = True)
data_user.head()

,time,count_all,count_sub,count_cus
0,2016-01-01,484933,484933,0
1,2016-02-01,531048,531048,0
2,2016-03-01,826678,826678,0
3,2016-04-01,882679,882679,0
4,2016-05-01,1035959,1035959,0


In [179]:
s2 = pd.Series(range(0,len(data_gender.index)))
data_gender.set_index([s2],inplace = True)
data_gender.head()

,time,count_all,count_m,count_f,count_u
0,2016-01-01,484933,379312,104457,1164
1,2016-02-01,531048,417215,112587,1246
2,2016-03-01,826678,634214,190551,1913
3,2016-04-01,882679,674127,206510,2042
4,2016-05-01,1035959,783687,249831,2441


In [181]:
s3 = pd.Series(range(0,len(data_age.index))) # give new ordered index
data_age.set_index([s3],inplace = True)
data_age.head()

,time,count_all,count_g1,count_g2,count_g3,count_g4,count_g5,count_g6,count_g7,count_g8
0,2016-01-01,484932,1707,6548,109880,157028,109291,74776,22729,2973
1,2016-02-01,531025,1875,12614,125913,167770,115646,79242,24657,3308
2,2016-03-01,826631,2780,18494,208283,260706,175571,118439,37196,5162
3,2016-04-01,882548,2822,20496,235703,280700,180095,120298,37163,5271
4,2016-05-01,1035719,3730,21318,280539,332884,212194,137856,41444,5754


### 4. Output data

In [182]:
data_user.to_csv('output/data_usertype.csv')
data_gender.to_csv('output/data_gender.csv')
data_age.to_csv('output/data_age.csv')